In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [21]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
import feedback.secret as secret
from feedback.database import open_connection
import openai

## Donors

In [13]:
db_name = "fr_new" 
username = secret.database_username 
password = secret.database_password 
ip_address = "localhost"
port = secret.database_port

In [14]:
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [15]:
start_date = "2024-05-20"
end_date = "2024-05-26"

In [16]:
info_file = pd.read_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date))

In [22]:
client = openai.OpenAI(api_key=secret.openai_api_key)

In [26]:
improve_instructions(client,cursor,info_file)

On 1 out of 9
{
    "explanation": "The volunteer's feedback indicates a crucial change in contact information for the recipient, Second Chance Center - Downtown. The previous contact person, Les, is no longer there, and the volunteer now uses his cell phone number for communication. Additionally, the volunteer suggests ringing the doorbell instead of contacting Les directly. These updates are vital for future rescue trips to ensure smooth communication and coordination.",
    "info_update": "Enter the store and let the staff know you are there to pick up a donation for We Don't Waste. Ring the doorbell for assistance. Contact Les at [new cell phone number] for further information."
}
On 2 out of 9
{
    "explanation": "The volunteer's feedback indicates a change in the recipient location and a significant increase in the amount of food received compared to the estimate. This information is crucial for future rescues to ensure accurate delivery to the correct location and proper handli

["Enter the store and let the staff know you are there to pick up a donation for We Don't Waste. Ring the doorbell for assistance. Contact Les at [new cell phone number] for further information.",
 'Take the food to the Healing Place instead of Beacon House. Received much more food than estimated to pick up from the elementary school. Go to kitchen door.',
 'Park at the front of the Neighbourhood House on West 7. Someone will be there to help you load your vehicle. For any issues, call Anthony at 778-896-6754. The pickup and drop-off locations need clarification to avoid confusion. Ensure clear instructions for both locations are provided to volunteers to prevent navigation errors. Additionally, ensure better communication backup plans in case of connectivity issues by providing alternative contact methods.',
 'Check in with donor for donuts donation. Drop off location is the Salvation Army in Long Beach-Citadel. Use Smart and Final on Atlantic in LB as a reference point.',
 'Check in 